<a href="https://colab.research.google.com/github/oxanaRC/CE902/blob/main/webscrap_bedrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ppd_data_jan_feb_2018.csv to ppd_data_jan_feb_2018 (1).csv
User uploaded file "ppd_data_jan_feb_2018.csv" with length 18870639 bytes


In [16]:
import pandas as pd
from pathlib import Path

In [17]:
fields=['unique_id','deed_date','postcode','saon','paon','street','property_type' ]
dfs = [pd.read_csv(csv_file, usecols=fields) for csv_file in uploaded]

In [18]:
results = pd.concat(dfs)
display(results)

,unique_id,deed_date,postcode,property_type,saon,paon,street
0,666758D6-D646-3363-E053-6B04A8C0D74E,26/01/2018,AL1 1AR,F,NaN,APARTMENT 3,OLD MILE HOUSE COURT
1,68FEB20B-7CD5-38DA-E053-6C04A8C051AE,23/02/2018,AL1 1HR,F,FLAT 12,"AVENTINE COURT, 101",HOLYWELL HILL
2,7011B10A-2C75-8ED6-E053-6B04A8C075C1,13/02/2018,AL1 1LA,O,FLAT 3,15,LONDON ROAD
3,7011B10A-2CF8-8ED6-E053-6B04A8C075C1,13/02/2018,AL1 1LA,O,NaN,15C,LONDON ROAD
4,6B32222C-4167-01F1-E053-6C04A8C0D2C8,23/02/2018,AL1 1LR,F,NaN,101,LONDON ROAD
...,...,...,...,...,...,...,...
142185,68FEB20C-6D95-38DA-E053-6C04A8C051AE,22/02/2018,NaN,O,GARAGE AT THE BACK OF,12,MALVERN ROAD
142186,6B32222D-107B-01F1-E053-6C04A8C0D2C8,24/01/2018,NaN,O,GARAGE,75,WHITEWOOD WAY
142187,68FEB20C-6467-38DA-E053-6C04A8C051AE,22/02/2018,NaN,O,NaN,UNIT 3,STAG BUSINESS PARK
142188,666758D7-B216-3363-E053-6B04A8C0D74E,31/01/2018,NaN,O,NaN,WREKIN GRAIN STORE,NaN


In [19]:
results['rm_url1'] =results['postcode'].str[0:3]+'-' +results['postcode'].str[4:8]+'.html?'
results['rm_url2'] =results['postcode'].str[0:3]+'-' +results['postcode'].str[4:8]+'.html?page=2'   
results['rm_url3'] =results['postcode'].str[0:2]+'-' +results['postcode'].str[3:7]+'.html?'
results['rm_url4'] =results['postcode'].str[0:2]+'-' +results['postcode'].str[3:7]+'.html?page=2'
results['rm_url5'] =results['postcode'].str[0:4]+'-' +results['postcode'].str[5:9]+'.html?'
results['rm_url6'] =results['postcode'].str[0:4]+'-' +results['postcode'].str[5:9]+'.html?page=2'


In [20]:
results_amend1=results.dropna(subset=['postcode'])
#print(results_amend1)

In [21]:
import requests
from bs4 import BeautifulSoup
from re import findall
import re
from pprint import pprint

In [29]:
# Example of files download
#with open('example.txt', 'w') as f:
#  f.write('some content')
#
#files.download('example.txt')

In [22]:
#df_1 =results_amend1.iloc[:300,:]
#[fromRow:toRow,:] 
#df_1 =results_amend1.iloc[301:1000,:]
#df_1 =results_amend1.iloc[1001:2000,:] 
#df_1 =results_amend1.iloc[2001:4000,:] 2K records 61min
#df_1 =results_amend1.iloc[4001:8000,:] 4K records 120min
#df_1 =results_amend1.iloc[8001:10001,:] 2K records 40min after optimising
#df_1 =results_amend1.iloc[10001:14000,:] #4K records 90 min
#df_1 =results_amend1.iloc[14001:40000,:] #26K records 
#df_1 =results_amend1.iloc[14001:20000,:] #6K records 120 min
#df_1 =results_amend1.iloc[20001:30000,:] #10K records 
#df_1 =results_amend1.iloc[30001:50001,:] #20K records 6 hours
df_1 =results_amend1.iloc[50001:55001,:] #5K records 
df_1

,unique_id,deed_date,postcode,property_type,saon,paon,street,rm_url1,rm_url2,rm_url3,rm_url4,rm_url5,rm_url6
50001,64342BFF-4C3D-422C-E053-6C04A8C0FB8A,09/01/2018,HP1 1BA,O,NaN,156,MARLOWES,HP1-1BA.html?,HP1-1BA.html?page=2,HP- 1BA.html?,HP- 1BA.html?page=2,HP1 -BA.html?,HP1 -BA.html?page=2
50002,64342BFF-4BED-422C-E053-6C04A8C0FB8A,09/01/2018,HP1 1BA,O,NaN,156,MARLOWES,HP1-1BA.html?,HP1-1BA.html?page=2,HP- 1BA.html?,HP- 1BA.html?page=2,HP1 -BA.html?,HP1 -BA.html?page=2
50003,666758D6-D476-3363-E053-6B04A8C0D74E,13/02/2018,HP1 1BE,F,FLAT 21,CAMDEN HOUSE,MARLOWES,HP1-1BE.html?,HP1-1BE.html?page=2,HP- 1BE.html?,HP- 1BE.html?page=2,HP1 -BE.html?,HP1 -BE.html?page=2
50004,666758D6-D6CC-3363-E053-6B04A8C0D74E,15/02/2018,HP1 1DG,F,9 THE PODIUM,MOOR END LODGE,COTTERELLS,HP1-1DG.html?,HP1-1DG.html?page=2,HP- 1DG.html?,HP- 1DG.html?page=2,HP1 -DG.html?,HP1 -DG.html?page=2
50005,64342BFE-3529-422C-E053-6C04A8C0FB8A,19/01/2018,HP1 1HP,S,NaN,164,LEIGHTON BUZZARD ROAD,HP1-1HP.html?,HP1-1HP.html?page=2,HP- 1HP.html?,HP- 1HP.html?page=2,HP1 -HP.html?,HP1 -HP.html?page=2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54996,666758D7-3E84-3363-E053-6B04A8C0D74E,05/01/2018,IP33 2QS,T,NaN,17,FALLOWFIELD WALK,IP3- 2QS.html?,IP3- 2QS.html?page=2,IP-3 2Q.html?,IP-3 2Q.html?page=2,IP33-2QS.html?,IP33-2QS.html?page=2
54997,666758D7-3FE5-3363-E053-6B04A8C0D74E,05/01/2018,IP33 2QS,T,NaN,34,FALLOWFIELD WALK,IP3- 2QS.html?,IP3- 2QS.html?page=2,IP-3 2Q.html?,IP-3 2Q.html?page=2,IP33-2QS.html?,IP33-2QS.html?page=2
54998,6B32222C-9D00-01F1-E053-6C04A8C0D2C8,29/01/2018,IP33 2SB,S,NaN,30,SEXTONS MEADOWS,IP3- 2SB.html?,IP3- 2SB.html?page=2,IP-3 2S.html?,IP-3 2S.html?page=2,IP33-2SB.html?,IP33-2SB.html?page=2
54999,64342BFE-C7FE-422C-E053-6C04A8C0FB8A,08/01/2018,IP33 2SN,D,NaN,6,BYFIELD WAY,IP3- 2SN.html?,IP3- 2SN.html?page=2,IP-3 2S.html?,IP-3 2S.html?page=2,IP33-2SN.html?,IP33-2SN.html?page=2


In [23]:
def findBedrooms(a_url, housenum1, housenum2):
    r = requests.get(a_url)
    c=r.content
    # Parse the html content
    soup = BeautifulSoup(c,"html.parser")
    #print('output from url ' + a_url)
    main_content = soup.find_all('script')
    content_string = str(main_content)
    r1= re.findall(r".address..................|.bedroom.....", content_string)
    #pprint(r1)
    r2=re.findall(r".address..................", content_string)    
    print_bedrooms="no"
    countNoOfFinds=0 
    my_address='not_found'
    my_bedrooms='not_found'
    for eachP in r1:  
      my_check0=eachP.lower().find('bedroom')
      #-----------------------
      if  my_check0==-1: 
        my_check1=eachP.lower().find(housenum1)
      else:
        my_check1=-1
      #-----------------------  
      r_test=re.findall('[0-9]+[a-z]?', eachP)
      if len(r_test)!=0:
         mycheck2=r_test[0] 
      else:
         mycheck2=0
      #-----------------------      
      if (my_check1!=-1 and mycheck2==housenum2 ) or print_bedrooms=="yes" :
          print_bedrooms="yes"  
          if my_check1==-1  :
            countNoOfFinds=countNoOfFinds+1 
            print_bedrooms="no"
            my_bedrooms=eachP
          else:
            my_address=eachP  
            
    return countNoOfFinds, my_address, my_bedrooms        
    print('-------------------------------') 
    print('countNoOfFinds '+ str(countNoOfFinds))
    print('-------------------------------')

In [ ]:
countIdentifiedRecords=0
list_for_bedrooms_table=[]
for row in df_1.itertuples():
    url1='https://www.rightmove.co.uk/house-prices/'+row[8]
    url2='https://www.rightmove.co.uk/house-prices/'+row[9]
    url3='https://www.rightmove.co.uk/house-prices/'+row[10]
    url4='https://www.rightmove.co.uk/house-prices/'+row[11]
    url5='https://www.rightmove.co.uk/house-prices/'+row[12]
    url6='https://www.rightmove.co.uk/house-prices/'+row[13]
    url_list = [url1, url2, url3, url4,url5, url6]
    bedrooms_table_new_row=[]
    propId=row[1] #unique identifier ppd_data
    house_num1=(str(row[5])+" " +str(row[6])).lower()
    if house_num1[0:3]=="nan":
        house_num1=house_num1[4:]       
    find_house_num2= re.findall('[0-9]+[a-z]?', house_num1) 
    if len(find_house_num2)!=0:
      house_num2=find_house_num2[0]
      #print(house_num2) #print 3
    else:
      house_num2 = 0
    bedrooms_table_new_row.append(propId)
    bedrooms_table_new_row.append(house_num1)
    bedrooms_table_new_row.append(house_num2)
    this_prop_num_of_finds=0
    this_prop_num_of_beds=-1
    for each_url in url_list:
      containsSpace = " " in each_url #for cheking if the url string contains spaces
      if not containsSpace:
        if this_prop_num_of_finds==0:
          countNoOfFinds, my_address, my_bedrooms=findBedrooms(each_url,house_num1,house_num2 )
          countIdentifiedRecords=countIdentifiedRecords + countNoOfFinds
          #----------------------------------------
        else:
          countNoOfFinds=0
          my_address="na" 
          my_bedrooms="na" 
        bedrooms_table_new_row.append(each_url)
        bedrooms_table_new_row.append(countNoOfFinds)        
        bedrooms_table_new_row.append(my_address)
        bedrooms_table_new_row.append(my_bedrooms)
        this_prop_num_of_finds=this_prop_num_of_finds+countNoOfFinds
        if countNoOfFinds>0:
          r_beds=re.findall('[0-9]+', my_bedrooms)
          if len(r_beds)!=0:
            this_prop_num_of_beds=r_beds[0]

    bedrooms_table_new_row.append(this_prop_num_of_finds)
    bedrooms_table_new_row.append(this_prop_num_of_beds)      
    list_for_bedrooms_table.append(bedrooms_table_new_row)  

In [ ]:
countIdentifiedRecords

In [ ]:
bedroomsDF = pd.DataFrame(list_for_bedrooms_table)
bedroomsDF.columns =['id', 'house_num1',  'house_num2','url1', 'num_of_finds1', 'matched_addr1', 'bedrooms1' ,'url2', 'num_of_finds2', 'matched_addr2', 'bedrooms2','overall_num_of_finds','num_of_beds']
bedroomsDF

In [ ]:
with open('example.csv', 'w') as f:
  #f.write('some content')
  bedroomsDF.to_csv('example.csv', index=False)
files.download('example.csv')  